# Clustering

Vamos a realizar operaciones de **clustering** (segmentación) de datos.
La idea es encontrar una estructura dentro de un dataset donde originalmente no la había.
No se tiene un objetivo de predicción (se trata **aprendizaje no supervisado**), sino de uno de entendimiento de los datos a través del particionamiento del dataset en grupos de instancias.

# Parte 1. K-Means con datos sintéticos

Para poder entender como se utilizan los algoritmos de clustering, vamos inicialmente a crear un dataset sintético con datos ficticios que nos permita ilustrar los aspectos de llamado a los métodos de python.

Vamos a generar y visualizar en un plot 300 puntos aleatorios distribuidos alrededor de 4 centros en un espacio bidimensional, con una desviación estándar de 0.7. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
from collections import Counter


from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, silhouette_samples, silhouette_score
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, grupo = make_blobs(n_samples=300, centers=4, cluster_std=0.8, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

En X van a quedar las coordenadas de los puntos y en **grupo** los clusters originales a los que pertencen los datos.

In [ ]:
X[0:5]

In [ ]:
grupo[0:5]

El algoritmo de K-Means recibe como parámetro el número de clusters que se buscan (hay que sepecificarlo ya que no lo determina automáticamente). Como sabemos que los datos sintéticos se crearon con 4 grupos, vamos a analizar si K-Means los logra detectar.

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(X)
grupo_kmeans = kmeans.predict(X)

Vamos a plotear los clusters encontrados con diferentes colores.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=grupo_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

Ahora comparemos los grupos encontrados con los reales, utilizando una matriz de confusión.

In [ ]:
cm = confusion_matrix(grupo, grupo_kmeans)
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title("Comparación entre los clusters reales y los descubiertos por K-Means")
plt.colorbar()
tick_marks = np.arange(4)
plt.xticks(tick_marks, ['0','1','2','3'])
plt.yticks(tick_marks, ['0','1','2','3'])
cm

En las filas encontramos los grupos reales y en las columnas los de K-Means. Encontramos 6 errores, todos asociados por k-means al grupo 0 cuando eran de alguno de los otros 3 grupos.

Hay que tener en cuenta que el orden de los nombres de los grupos generados puede no conincidir con el orden de los grupos encontrados por K-Means, como es el caso aquí.

Lo que vemos es que parece haber una concordancia entre los clusters encontrados por K-Means y los reales: los grupos 0, 1, 2 y 3 de k-means corresponden a los grupos 1, 0, 2, y 3 encontrados por K-Means, respectivamente.

Vamos a cambiar el orden de los clusters de k-means para poder entender mejor los resultados

In [ ]:
traducir = [1, 0, 2, 3]

grupo_kmeans_reorg = []
for g_k, g in zip(grupo_kmeans, grupo):
    grupo_kmeans_reorg.append(traducir[g_k])
print(grupo_kmeans_reorg)

Podemos hacer esto de una manera mas breve utilizando una de las particularidades de Python: List comprehensions, que permite resumir operaciones simples realizadas dentro de un ciclo:

In [ ]:
grupo_kmeans_reorg = [traducir[g_k] for g_k in grupo_kmeans]
print(grupo_kmeans_reorg)

In [ ]:
cm = confusion_matrix(grupo, grupo_kmeans_reorg)
cm

In [ ]:
accuracy_score(grupo, grupo_kmeans_reorg)

Veamos gráficamente cuáles son los registros que se asocian a un grupo diferente a su original.

In [ ]:
diferentes = []
for (x0, x1), g, gk in zip(X, grupo, grupo_kmeans_reorg):
    if g!=gk:
        diferentes.append([x0, x1])
        

In [ ]:
X[0:5]

In [ ]:
diferentes = np.array(diferentes)

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(X[:, 0], X[:, 1], c=grupo_kmeans_reorg, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

plt.scatter(diferentes[:, 0], diferentes[:, 1], c='red', marker="x", s=150)

**Preguntas**:
Comparamos los grupos creados por el clustering con los originales (esto se llama "clasificación no supervisada")
1. Expliquen la primera matriz de confusión obtenida y el por qué fue necesario recodificar los segmentos obtenidos por el clustering.
2. ¿Qué tan bien puede K-Means encontrar las categorías originales en terminos de accuracy?
3. ¿Tiene sentido crear un test set para un clustering?

# Parte 2. K-Means con datos reales

In [ ]:
data = pd.read_csv('01 - ComprasClientes.csv', na_values=".")
print(data.shape)
data.head(5)

In [ ]:
data.describe(include="all")

In [ ]:
data.info()

## Preparación de los datos

**¿Qué problemas saltan a la vista al inspeccionar los datos? Arréglenlos. **

...

...

...

...

## Clustering

**Con un k de 3, realice un clustering por K-Means (utilicen random_state=0).**

**Agregue una columna "Cluster" con el segmento correspondiente (0, 1, o 2) al dataset.**

**Con un k de 3, realice un clustering por K-Means (utilicen random_state=0).**

**Agregue una columna al dataset**


El método KMeans en scikit-learn permite definir los valores de ciertos parámetros que controlan la ejecución del algoritmo de clustering. Nos interesan particularmente:
- **n_clusters**: número de clusters que se desean (el parámetro "K"). Por defecto es 8.
- **init**: el método de inicialización de los centroides. Por defecto es "k-means++". Otros valores son "random" o un array con los centroides iniciales
- **n_init**: número de inicializaciones diferentes a ensayar para evitar llegar a un óptimo local. Por defecto es 10
- **max_iter**: Máximo número de iteraciones que se esparará para llegar a convergencia. Por defecto es 300.
- **tol**: tolerancia para determinar que se ha llegado o no a convergenia con respecto a la reducción del WSS (interia). Por defecto es 0.0001
- **random_state**: semilla de inicialización del generador pseudo-aleatorio para poder reproducir los resultados.

El objeto resultado del KMeans después de lanzado el ajuste del algoritmo consta de diferentes valores de salida:
- **cluster_centers_**: los centroides finales de los clusters.
- **labels_**: los clusters a los cuales termina perteneciendo cada instancia del set de aprendizaje.
- **interia_**: el WSS final.
- **n_iter_**: el número de iteraciones que tomó llegar a convergencia.

Podemos utilizar el objeto kmeans resultante como modelo de clasificación, al que a través del método *predict* se le puede enviar un dataset para evaluar y obtener los clusters a los que pertenecen.

...

...

...

...

...